In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

In [ ]:
df = pd.read_csv('../input/train.csv', sep=',')
df = df.reset_index(drop=True)
df.head()

Доля совпадающих пар

In [ ]:
df.is_duplicate.mean()

In [ ]:
df.shape

<h2>Bag of words</h2>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
df = df.dropna(how='any')
df.shape

Как видим, в выборке есть две строки с пропущенными полями, эти строки надо удалить

Построим bag of words по наборам подряд идущих символов

In [ ]:
BOW = CountVectorizer(max_df=0.999, min_df=1000, max_features=None, 
                                      analyzer='char', ngram_range=(1,2), 
                                      binary=True, lowercase=True)

Обучаем bag of word на всех вопросах из трейновой выборки

In [ ]:
%%time
BOW.fit(pd.concat((df.loc[:,'question1'],df.loc[:,'question2'])).unique())

И считаем отдельно матрицу bag of words для столбцов с первым и со вторым вопросом

In [ ]:
%%time
BOW_1 = BOW.transform(df.loc[:,'question1'])
BOW_2 = BOW.transform(df.loc[:,'question2'])
labels = df.is_duplicate.values

Если в обоих вопросах данная последовательность символов в одинаковом состоянии (либо и там, и там присутствует, либо нигде нет), то в Х будет стоять 0, если в разных состояниях, то -1

In [ ]:
X = -(BOW_1 != BOW_2).astype(int)
y = labels
X.shape, y.shape

Используем лог. регрессию. Веса нужны для того, чтобы скомпенсировать разную долю дублирующихся вопросов в тестовой и трейновой выборке. В трейне 0,37, в тесте 0,17, значит надо умножить долю класса 1 на 0,46 - тогда из 0,37 трейновых он превратится в 0,17 тестовых. Соответственно долю класса 0 умножаем на 1,32 (с 0,63 до 0,83)

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=0.1, solver='sag', class_weight={1: 0.46, 0: 1.32}).fit(X, y)

Готовим тестовые данные. Для экономии памяти пишем в переменные с теми же именами

In [ ]:
%%time
filepath = '../input/test.csv'
df = pd.read_csv(filepath)
df.loc[df['question1'].isnull(),['question1','question2']] = 'random empty question'
df.loc[df['question2'].isnull(),['question1','question2']] = 'random empty question'

BOW_1 = BOW.transform(df.loc[:,'question1'])
BOW_2 = BOW.transform(df.loc[:,'question2'])

X = -(BOW_1 != BOW_2).astype(int)

Тестовые данные почти в 6 раз объемнее трейновых

In [ ]:
X.shape

In [ ]:
%%time
scores = model.predict_proba(X)

In [ ]:
df_out = pd.DataFrame()
df_out['test_id'] = df['test_id']
df_out['is_duplicate'] = scores[:,1]
df_out.to_csv('submission.csv', index=False)

Этот код генерирует решение со скором 0.36967, но если поменять параметры в CountVectorizer(n_grams=(1,3), max_features=None), то получим скор 0.33754. При этом считаться будет чуть меньше часа, а полное число признаков в Х будет 3556.